In [ ]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
# Import the requests library.
import requests
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime
# Import the API key.
from config import weather_api_key

In [ ]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

In [ ]:
#Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [ ]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
#coordinates

In [ ]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

In [ ]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [10]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cape Town,-33.9258,18.4232,61.09,86,75,8.01,ZA,2021-11-01 10:06:55
1,Faanui,-16.4833,-151.7500,78.94,74,26,14.38,PF,2021-11-01 10:07:01
2,Yatou,37.1500,122.3833,50.27,52,0,10.69,CN,2021-11-01 10:05:05
3,Vaini,-21.2000,-175.2000,75.36,94,20,5.75,TO,2021-11-01 10:09:53
4,Rikitea,-23.1203,-134.9692,72.05,73,100,13.94,PF,2021-11-01 10:06:58
5,Khatanga,71.9667,102.5000,5.05,96,100,10.69,RU,2021-11-01 10:06:58
6,Dikson,73.5069,80.5464,5.49,93,72,10.20,RU,2021-11-01 10:08:41
7,Buala,-8.1450,159.5921,79.47,83,100,7.63,SB,2021-11-01 10:12:15
8,Ingham,42.6000,-84.3833,37.26,84,1,0.00,US,2021-11-01 10:12:16
9,Hermanus,-34.4187,19.2345,57.79,87,62,26.44,ZA,2021-11-01 10:10:42


In [13]:
new_column_order=['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed']
city_data_df=city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Cape Town,ZA,2021-11-01 10:06:55,-33.9258,18.4232,61.09,86,75,8.01
1,Faanui,PF,2021-11-01 10:07:01,-16.4833,-151.7500,78.94,74,26,14.38
2,Yatou,CN,2021-11-01 10:05:05,37.1500,122.3833,50.27,52,0,10.69
3,Vaini,TO,2021-11-01 10:09:53,-21.2000,-175.2000,75.36,94,20,5.75
4,Rikitea,PF,2021-11-01 10:06:58,-23.1203,-134.9692,72.05,73,100,13.94
5,Khatanga,RU,2021-11-01 10:06:58,71.9667,102.5000,5.05,96,100,10.69
6,Dikson,RU,2021-11-01 10:08:41,73.5069,80.5464,5.49,93,72,10.20
7,Buala,SB,2021-11-01 10:12:15,-8.1450,159.5921,79.47,83,100,7.63
8,Ingham,US,2021-11-01 10:12:16,42.6000,-84.3833,37.26,84,1,0.00
9,Hermanus,ZA,2021-11-01 10:10:42,-34.4187,19.2345,57.79,87,62,26.44


In [14]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# Loop through all the cities in our list.
for i in range(len(cities)):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + cities[i]